In [2]:

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os


2022-08-06 17:33:49.252678: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-06 17:33:49.252780: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:

prototxtPath=os.path.sep.join([r'deploy.prototxt'])
weightsPath=os.path.sep.join([r'res10_300x300_ssd_iter_140000.caffemodel'])



In [4]:
prototxtPath

'deploy.prototxt'

In [5]:
weightsPath

'res10_300x300_ssd_iter_140000.caffemodel'

In [6]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)
net

< cv2.dnn.Net 0x7f1636341f90>

In [7]:
model=load_model(r'mobilenet_v2.model')


2022-08-06 17:34:26.193755: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-06 17:34:26.193914: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (harsh-Inspiron-3576): /proc/driver/nvidia/version does not exist
2022-08-06 17:34:26.211322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
image=cv2.imread(r'example_01.png')

In [9]:
image

array([[[186, 183, 145],
        [186, 184, 144],
        [186, 184, 144],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       [[184, 183, 145],
        [184, 183, 145],
        [184, 183, 145],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       [[182, 181, 146],
        [182, 181, 146],
        [183, 182, 144],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       ...,

       [[ 90,  87,  77],
        [ 90,  87,  77],
        [ 91,  88,  78],
        ...,
        [106, 102,  96],
        [106, 102,  96],
        [106, 102,  96]],

       [[ 90,  87,  77],
        [ 91,  88,  78],
        [ 91,  88,  78],
        ...,
        [105, 101,  95],
        [105, 101,  95],
        [105, 101,  95]],

       [[ 90,  87,  77],
        [ 91,  88,  78],
        [ 91,  88,  78],
        ...,
        [104, 100,  94],
        [104, 100,  94],
        [104, 100,  94]]

In [10]:
(h,w)=image.shape[:2]
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [11]:
net.setInput(blob)
detections=net.forward()

In [ ]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
    
    
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 [==============================] - 3s 3s/step
